In [1]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier
from Standardising import standard
from Recoding_SIC_Codes import Industry_Division 
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support


# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
ipo_data.I3 = ipo_data.I3.astype(object) # Converting to object for the moment to tidy up summary statistics
# Run Descriptive Statistics on Data File
#print(ipo_data.dtypes)
# Replacing NaN in C2 with 1 
ipo_data['C2']=ipo_data['C2'].fillna(1)


# Removing negative and 0 values for the word/sentence counts as it is beleived that these are errors
# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].mean())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].mean())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].mean())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].mean())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].mean())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].mean())

# Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())


C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\Tim\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

# Creating our target and control variables
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)

#Creating ratios for the words, sentences and postive/negative counts. Ratios provide insight
calc(ipo_data)

#Removing redundent data
del ipo_data['C3']
del ipo_data['C5']
del ipo_data['C6']
# del ipo_data['T1']
# del ipo_data['T2']
del ipo_data['T3']
del ipo_data['T4']
del ipo_data['T5']
del ipo_data['S1']
del ipo_data['S2']
del ipo_data['S3']
del ipo_data['P(1Day)']
del ipo_data['P(L)']
del ipo_data['P(H)']
del ipo_data['P(IPO)']
#Looking at the new description of the data
ipo_data.describe()

,C1,C2,C4,C7,T1,T2,Y1,Y2,C3x,C6x,C5x,T3x,T4x,T5x,S1x,S2x,S3x
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,149.728788,0.863636,0.007282,500.459962,465.634361,12796.076671,0.513196,0.708211,0.536657,0.052708,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854
std,150.328785,0.343426,0.032775,1558.767222,175.612567,5401.439089,0.500193,0.454919,0.499020,0.107639,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970
min,10.000000,0.000000,-0.162352,0.074000,132.000000,527.000000,0.000000,0.000000,0.000000,0.000000,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125
25%,85.000000,1.000000,-0.013352,41.511500,351.250000,9213.250000,0.000000,0.000000,0.000000,0.000000,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570
50%,110.000000,1.000000,0.007282,134.738000,444.500000,12064.000000,1.000000,1.000000,1.000000,0.000000,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305
75%,152.750000,1.000000,0.030941,500.459962,550.750000,15240.250000,1.000000,1.000000,1.000000,0.071429,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883
max,2087.000000,1.000000,0.092896,30683.000000,1750.000000,49056.000000,1.000000,1.000000,1.000000,1.000000,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397


In [3]:
# Converting C6x and C2 to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)


sqrt_transform=['C6x','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation

ipo_data['T5xbin'] = 0
# ipo_data.loc[(ipo_data['T5x'] >= 0.080692),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= 0) & (ipo_data['T5x'] < .0474),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= .047401) & (ipo_data['T5x'] < .0519), 'T5xbin'] = 2
ipo_data.loc[(ipo_data['T5x'] >= .051901) & (ipo_data['T5x'] < .05647), 'T5xbin'] = 3
ipo_data.loc[ipo_data[ 'T5x'] >= .0564701,'T5xbin'] = 4

ipo_data['T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] < 0.897133),'T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] >= 0.897133),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= 0.703925) & (ipo_data['T3x'] < 0.883578),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= .883578) & (ipo_data['T3x'] < 0.897133), 'T3xbin'] = 2
ipo_data.loc[(ipo_data['T3x'] >= .897133) & (ipo_data['T3x'] < 0.909732), 'T3xbin'] = 3
ipo_data.loc[ipo_data[ 'T3x'] >= 0.909732,'T3xbin'] = 4

print(ipo_data.dtypes) # checking the types
print(ipo_data.T5xbin.value_counts())

I1         object
I2         object
I3         object
C1        float64
C2          int32
C4        float64
C7        float64
T1        float64
T2        float64
Y1          int64
Y2          int64
C3x         int64
C6x       float64
C5x       float64
T3x       float64
T4x       float64
T5x       float64
S1x       float64
S2x       float64
S3x       float64
T5xbin      int64
T3xbin      int64
dtype: object
4    171
2    171
3    170
1    170
Name: T5xbin, dtype: int64


In [4]:
"""Code to recode the column I3 into industry level.

    Version Control:
    Initial coding
    ------------------------------------------
    Date 8-Feb-18, Author: Danielle Ezzo, Desc: Initial Coding
    """

def Industry_Division1(row,method):
    # Recoding based on table in milestone report
    if method==1:
        if (row['I3']> 99 and row['I3']< 1000):
            return 'Agriculture, Forestry and Fishing'
        elif (row['I3']> 999 and row['I3']< 1500):
            return  'Mining'
        elif (row['I3']> 1499 and row['I3']< 1800):
            return 'Construction'
        elif (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Transportation, Communications, Electric, Gas and Sanitary service'
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Wholesale Trade'
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Retail Trade'
        elif (row['I3']> 5999 and row['I3']< 6800):
            return 'Finance, Insurance and Real Estate'
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        elif (row['I3']> 9099 and row['I3']< 9730):
            return 'Public Administration'
        elif (row['I3']> 9899 and row['I3']< 10000):
            return 'Nonclassifiable'
        else:
            return 'Missing'
    elif method==2:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        else:
            return 'Other'
    elif method==3:
        if (row['I3']> 7369 and row['I3']< 7375) :
            return 'Tech'
        else:
            return 'NonTech'
    elif method==4:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing' # Manufacturing
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' # Tech Comms etc Services
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services' #Services (combining with above)
        else:
            return 'Other' # All Other
    elif method==5:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' #TCEGS
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==6:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'TCEGS' 
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==7:
        if (row['I3']> 99 and row['I3']< 5000):
            return 'Manual Labor'
        else: 
            return 'NonManual Labor'
        

In [5]:
# Add the new data to the end of the table
ipo_data.I3 = ipo_data.I3.astype(int) # recoding to correct type

# import functools
# import operator


# applying our function to recode our industry codes to sector level
ipo_data['IndDivision'] = ipo_data.apply(Industry_Division1,method=2, axis=1) 
print(ipo_data.head(5))

     I1                           I2    I3        C1  C2        C4        C7  \
0    RA         RAILAMERICA INC /DE   4011  4.343805   1  0.005264  6.231398   
1     G                 Genpact LTD   8742  4.418841   1 -0.053282  6.426451   
2  HURN  Huron Consulting Group Inc.  8742  5.030438   1  0.024642  4.703150   
3  ICFI     ICF International, Inc.   8742  4.941642   1  0.051895  5.177381   
4  SRVY       GREENFIELD ONLINE INC   8742  4.553877   1 -0.050492  3.253007   

      T1       T2  Y1     ...            C5x       T3x       T4x       T5x  \
0  463.0  13494.0   1     ...       2.470315  0.865866  0.710583  0.043871   
1  641.0  15238.0   1     ...       5.848265  0.905303  0.561622  0.047710   
2  314.0   9689.0   0     ...       3.257192  0.874290  0.719745  0.052843   
3  453.0  11082.0   1     ...       2.772191  0.909673  0.576159  0.051886   
4  409.0  10417.0   1     ...       3.292713  0.899587  0.650367  0.048094   

        S1x       S2x       S3x  T5xbin  T3xbin  I

In [6]:
dummies = pd.get_dummies(ipo_data['IndDivision'])

ipo_data = ipo_data.join(dummies)
ipo_data.head(5)

,I1,I2,I3,C1,C2,C4,C7,T1,T2,Y1,...,T4x,T5x,S1x,S2x,S3x,T5xbin,T3xbin,IndDivision,Manufacturing,Other
0,RA,RAILAMERICA INC /DE,4011,4.343805,1,0.005264,6.231398,463.0,13494.0,1,...,0.710583,0.043871,0.082570,0.122955,0.081668,1,1,Other,0,1
1,G,Genpact LTD,8742,4.418841,1,-0.053282,6.426451,641.0,15238.0,1,...,0.561622,0.047710,0.077702,0.086114,0.109887,2,3,Other,0,1
2,HURN,Huron Consulting Group Inc.,8742,5.030438,1,0.024642,4.703150,314.0,9689.0,0,...,0.719745,0.052843,0.055644,0.070385,0.090297,3,1,Other,0,1
3,ICFI,"ICF International, Inc.",8742,4.941642,1,0.051895,5.177381,453.0,11082.0,1,...,0.576159,0.051886,0.075398,0.081162,0.126022,2,3,Other,0,1
4,SRVY,GREENFIELD ONLINE INC,8742,4.553877,1,-0.050492,3.253007,409.0,10417.0,1,...,0.650367,0.048094,0.081974,0.095998,0.109543,2,3,Other,0,1


In [7]:
def standard12(data,method):
    """Standarising data using various methods.
    
    Method 1 is MinMax scaling 
    
    Version Control:
    Initial coding
    ------------------------------------------
    Date 4-Feb-18, Author: Conor Feeney, Desc: Initial Coding
    """
    if method == 1:
        X_std = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
        data = X_std * (1 - 0) + 0
        return data
    elif method==2:
        data = (data)/(10**len(str(int(max(data)))))
        return data
    elif method==3:
        return data

In [8]:
def outlier1(data,method):
    """Deals with outliers in pandas series using differeing methods.
    Only valid for pandas series
    Method 1 is calculating the points that are greater than or less than 3 standard deviations
    away from the mean and setting any values outside this band to be the upper / lower bound 
    (mean+/- 3*standard deviation)
    Method 2 is calculating the points that are greater than or less than 3 standard deviations
    away from the mean and setting any values outside this band to be the mean
    Method 3 is calculating the IQR and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
    and setting to be the mean
    Method 4 is calculating the IQR and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
    and setting to be the the Q1 or Q3 respectively.
    
    Version Control:
    Initial coding
    ------------------------------------------
    Date 4-Feb-18, Author: Conor Feeney, Desc: Initial Coding
    """

    if method == 1:
        mean, std = data.mean(),data.std() # Creating mean and standard dev variables

        #Calculation
        outlier_lower = data< mean-3*std
        outlier_upper = data> mean+3*std
        data[outlier_lower] = mean-3*std
        data[outlier_upper] = mean+3*std
        #print('Method: {}'.format(method))
        return data
    elif method ==2:
        mean, std = data.mean(),data.std()# Creating mean and standard dev variables

        #Calculation
        outlier_lower = data< mean-3*std
        outlier_upper = data> mean+3*std
        data[outlier_lower] = mean
        data[outlier_upper] = mean
        #print('Method: {}'.format(method))
        return data
    elif method == 3:
        # Creating mean and quartile variables
        mean= data.mean()
        lq=data.quantile(0.25)
        uq=data.quantile(0.75)
        iqr = (data.quantile(0.75)- data.quantile(0.25))*1.5

        # Calculation
        outlier_lower = data< lq-iqr
        outlier_upper = data> uq+iqr
        data[outlier_lower] = mean
        data[outlier_upper] = mean
        #print('Method: {}'.format(method))
        return data
    elif method ==4:
        # Creating mean and quartile variables
        lq=data.quantile(0.25)
        uq=data.quantile(0.75)
        iqr = (data.quantile(0.75)- data.quantile(0.25))*1.5

        #Calculation
        outlier_lower = data< lq-iqr
        outlier_upper = data> uq+iqr
        data[outlier_lower] = lq
        data[outlier_upper] = uq
        #print('Method: {}'.format(method))
        return data
    elif method ==5:
        return data

In [9]:
for i in (['T3x','T5x','C6x']):
    outlier(ipo_data[i],4)
# outlier(ipo_data['T3x'],4) 
# outlier(ipo_data['T5x'],4) 
# outlier(ipo_data['C6x'],4) 
ipo_data.describe()

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


,I3,C1,C2,C4,C7,T1,T2,Y1,Y2,C3x,...,T3x,T4x,T5x,S1x,S2x,S3x,T5xbin,T3xbin,Manufacturing,Other
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,4851.573314,4.765815,0.863636,0.007282,4.748030,465.634361,12796.076671,0.513196,0.708211,0.536657,...,0.896026,0.629876,0.052022,0.072485,0.094300,0.107151,2.501466,2.500000,0.451613,0.548387
std,2065.497876,0.660942,0.343426,0.032775,1.965899,175.612567,5401.439089,0.500193,0.454919,0.499020,...,0.018913,0.076265,0.006452,0.015820,0.016951,0.019313,1.118854,1.120166,0.498018,0.498018
min,100.000000,2.302585,0.000000,-0.162352,-2.603690,132.000000,527.000000,0.000000,0.000000,0.000000,...,0.844359,0.395480,0.034592,0.037268,0.046394,0.071590,1.000000,1.000000,0.000000,0.000000
25%,3175.250000,4.442651,1.000000,-0.013352,3.725969,351.250000,9213.250000,0.000000,0.000000,0.000000,...,0.883579,0.580815,0.047418,0.064454,0.083854,0.097826,2.000000,1.250000,0.000000,0.000000
50%,3881.000000,4.700480,1.000000,0.007282,4.903310,444.500000,12064.000000,1.000000,1.000000,1.000000,...,0.897133,0.629077,0.051901,0.071197,0.092735,0.106324,2.500000,2.500000,0.000000,1.000000
75%,7371.000000,5.028799,1.000000,0.030941,6.215528,550.750000,15240.250000,1.000000,1.000000,1.000000,...,0.909717,0.674852,0.056467,0.079108,0.102638,0.113504,3.750000,3.750000,1.000000,1.000000
max,8742.000000,7.643483,1.000000,0.092896,10.331464,1750.000000,49056.000000,1.000000,1.000000,1.000000,...,0.945481,1.443089,0.069380,0.348485,0.301797,0.483111,4.000000,4.000000,1.000000,1.000000


In [10]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,4):
        x=pd.DataFrame.copy(ipo_data)
        for i in (['C3x','C1','T4x','S1x','S2x','S3x','C2','T2','T1']): 
            outlier(x[i],j) 
            for p in range(len(x.columns)):
                if x.iloc[:,p].dtype == float:   
                    x.iloc[:,p]=standard12(x.iloc[:,p],m)
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other']
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y2']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y2'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            oversample = pd.concat([X_train,y_train],axis=1)
            max_size = oversample['Y2'].value_counts().max()

            lst = [oversample]
            for class_index, group in oversample.groupby('Y2'):
                lst.append(group.sample(max_size-len(group), replace=True))
            X_train = pd.concat(lst)
            y_train=pd.DataFrame.copy(X_train['Y2'])
            del X_train['Y2']
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.561577
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.53; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561548
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.51; AUC 0.49 
Optimization terminated successfully.
         Current function value: 0.561272
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.51; AUC 0.49 
Optimization terminated successfully.
         Current function value: 0.560903
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


         Current function value: 0.600473
         Iterations: 35
Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.52; AUC 0.49 
         Current function value: 0.600891
         Iterations: 35
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.54; AUC 0.51 
         Current function value: 0.600407
         Iterations: 35


C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

Optimization terminated successfully.
         Current function value: 0.561128
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.560959
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.53; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.560907
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561137
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.52; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.560562
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.51; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.560537
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.

C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Outlier Method 5; Standardise Method 2 Kmeans 3
F1 0.56; AUC 0.50 
         Current function value: 0.600407
         Iterations: 35
Outlier Method 5; Standardise Method 2 Kmeans 4
F1 0.59; AUC 0.53 
         Current function value: 0.601653
         Iterations: 35
Outlier Method 5; Standardise Method 2 Kmeans 5
F1 0.58; AUC 0.51 


C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 0.560480
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.53; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.560405
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.52; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.560479
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 4
F1 0.53; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.560406
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 5
F1 0.54; AUC 0.52 


In [11]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,4):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard12(x.iloc[:,i],m)
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other']
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y1']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y1'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
           
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.660565
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.56; AUC 0.56 
True
Optimization terminated successfully.
         Current function value: 0.665215
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.57; AUC 0.57 
True
Optimization terminated successfully.
         Current function value: 0.663015
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.56; AUC 0.56 
True
Optimization terminated successfully.
         Current function value: 0.665208
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.57 
True


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.660902
         Iterations 6
Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.660973
         Iterations 6
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.50; AUC 0.50 
Optimization terminated successfully.
         Current function value: 0.661012
         Iterations 6
Outlier Method 1; Standardise Method 2 Kmeans 4
F1 0.50; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.661801
         Iterations 6
Outlier Method 1; Standardise Method 2 Kmeans 5
F1 0.51; AUC 0.52 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


...

Optimization terminated successfully.
         Current function value: 0.659353
         Iterations 5
Outlier Method 4; Standardise Method 1 Kmeans 2
F1 0.57; AUC 0.57 
True
Optimization terminated successfully.
         Current function value: 0.664165
         Iterations 5
Outlier Method 4; Standardise Method 1 Kmeans 3
F1 0.55; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.661658
         Iterations 5
Outlier Method 4; Standardise Method 1 Kmeans 4
F1 0.55; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.664182
         Iterations 5
Outlier Method 4; Standardise Method 1 Kmeans 5
F1 0.52; AUC 0.53 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.663947
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 2
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.663940
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 3
F1 0.51; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.663625
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 4
F1 0.50; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.665098
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 5
F1 0.51; AUC 0.52 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.664763
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.50; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.663817
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.53; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.664968
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.665110
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.54; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.660196
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.660946
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.

In [12]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,4):
    for m in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard12(x.iloc[:,i],j)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],m) 
                    
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3x','T4x','T5x','S1x','S2x','S3x','C2','T2','T1','C6x','Other']
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y2']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y2'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            oversample = pd.concat([X_train,y_train],axis=1)
            max_size = oversample['Y2'].value_counts().max()

            lst = [oversample]
            for class_index, group in oversample.groupby('Y2'):
                lst.append(group.sample(max_size-len(group), replace=True))
            X_train = pd.concat(lst)
            y_train=pd.DataFrame.copy(X_train['Y2'])
            del X_train['Y2']
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(m,j,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.560702
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.50; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561548
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.51; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.561647
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.51; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561552
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.52; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.560097
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 2
F1 0.52; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.561073
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 3
F1 0.52; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.561182
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 4
F1 0.51; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.560939
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 5
F1 0.51; AUC 0.53 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


...

Optimization terminated successfully.
         Current function value: 0.559842
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 2
F1 0.56; AUC 0.57 
True
Optimization terminated successfully.
         Current function value: 0.560337
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 3
F1 0.51; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.560135
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 4
F1 0.56; AUC 0.56 
True
Optimization terminated successfully.
         Current function value: 0.560309
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 5
F1 0.55; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.560975
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 2
F1 0.53; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.561159
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 3
F1 0.52; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.561142
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561159
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 5
F1 0.52; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.560997
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.53; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.560957
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.54; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.560752
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.561044
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.53; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.560468
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.51; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.560339
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.

In [13]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,4):
    for m in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard12(x.iloc[:,i],j)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],m) 
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other']
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y1']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y1'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
           
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(m,j,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.664723
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.51; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.661339
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.50; AUC 0.50 
Optimization terminated successfully.
         Current function value: 0.659805
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.51; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.664949
         Iterations 5
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.50; AUC 0.50 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.662652
         Iterations 5
Outlier Method 2; Standardise Method 1 Kmeans 2
F1 0.51; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.662801
         Iterations 5
Outlier Method 2; Standardise Method 1 Kmeans 3
F1 0.50; AUC 0.50 
Optimization terminated successfully.
         Current function value: 0.661494
         Iterations 5
Outlier Method 2; Standardise Method 1 Kmeans 4
F1 0.51; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.666315
         Iterations 5
Outlier Method 2; Standardise Method 1 Kmeans 5
F1 0.51; AUC 0.51 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


...

Optimization terminated successfully.
         Current function value: 0.664803
         Iterations 5
Outlier Method 2; Standardise Method 3 Kmeans 2
F1 0.53; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.663736
         Iterations 5
Outlier Method 2; Standardise Method 3 Kmeans 3
F1 0.55; AUC 0.55 
True
Optimization terminated successfully.
         Current function value: 0.664687
         Iterations 5
Outlier Method 2; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.664854
         Iterations 5
Outlier Method 2; Standardise Method 3 Kmeans 5
F1 0.51; AUC 0.52 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.665435
         Iterations 5
Outlier Method 3; Standardise Method 3 Kmeans 2
F1 0.49; AUC 0.50 
Optimization terminated successfully.
         Current function value: 0.666487
         Iterations 5
Outlier Method 3; Standardise Method 3 Kmeans 3
F1 0.49; AUC 0.50 
Optimization terminated successfully.
         Current function value: 0.666493
         Iterations 5
Outlier Method 3; Standardise Method 3 Kmeans 4
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.666234
         Iterations 5
Outlier Method 3; Standardise Method 3 Kmeans 5
F1 0.51; AUC 0.52 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.664763
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.50; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.663817
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.53; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.664968
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.51; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.665110
         Iterations 5
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.54; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.661101
         Iterations 5
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.50; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.662378
         Iterations 5
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.